In [1]:
import os

from dotenv import load_dotenv, find_dotenv

In [2]:
load_dotenv(find_dotenv(), override=True)

True

In [5]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

### Loading Documents

In [3]:
def load_document(file):
    import os
    name, extension = os.path.splitext(file)

    if extension == '.pdf':
        from langchain.document_loaders import PyPDFLoader

        print(f'Loading {file}')

        loader = PyPDFLoader(file)
    elif extension == '.docx':
        from langchain.document_loaders import Docx2txtLoader

        print(f'Loading {file}')

        loader = Docx2txtLoader(file)
    elif extension == '.txt':
        from langchain.document_loaders import TextLoader

        loader = TextLoader(file)
    else:
        print('Document format is not supported!')

        return None

    data = loader.load()

    return data  

In [23]:
def load_from_wikipedia(query, lang='en', load_max_docs=2):
    from langchain.document_loaders import WikipediaLoader

    loader = WikipediaLoader(query=query, lang=lang, load_max_docs=load_max_docs)
    data = loader.load()

    return data

### Chunking Data

In [13]:
def chunk_data(data, chunk_size=256):
    from langchain.text_splitter import RecursiveCharacterTextSplitter

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)
    chunks = text_splitter.split_documents(data)

    return chunks    

### Calculating Cost

In [4]:
def print_embedding_cost(texts):
    import tiktoken

    enc = tiktoken.encoding_for_model('text-embedding-3-small')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    # check prices here: https://openai.com/pricing
    print(f'Total Tokens: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens * 0.02:.6f}')

### Embedding and Uploading to a Vector Database (Pinecone)

In [6]:
def insert_or_fetch_embeddings(index_name, chunks):
    import pinecone
    from langchain_community.vectorstores import Pinecone
    from langchain_openai import OpenAIEmbeddings
    from pinecone import ServerlessSpec

    pc = pinecone.Pinecone()
        
    embeddings = OpenAIEmbeddings(model='text-embedding-3-small', dimensions=1536)

    # loading from existing index
    if index_name in pc.list_indexes().names():
        print(f'Index {index_name} already exists. Loading embeddings ... ', end='')
        vector_store = Pinecone.from_existing_index(index_name, embeddings)
        print('Ok')
    else:
        print(f'Creating index {index_name} and embeddings ...', end='')

        pc.create_index(
            name=index_name,
            dimension=1536,
            metric='cosine',
            spec=ServerlessSpec(
                cloud="aws",
                region="us-east-1"
            ) 
        )

        vector_store = Pinecone.from_documents(chunks, embeddings, index_name=index_name)
        print('Ok')
        
    return vector_store
    

In [7]:
def delete_pinecone_index(index_name='all'):
    import pinecone
    pc = pinecone.Pinecone()
    
    if index_name == 'all':
        indexes = pc.list_indexes().names()
        print('Deleting all indexes ... ')
        for index in indexes:
            pc.delete_index(index)
        print('Ok')
    else:
        print(f'Deleting index {index_name} ...', end='')
        pc.delete_index(index_name)
        print('Ok')
    

### Asking and Getting Answers

In [8]:
def ask_and_get_answer(vector_store, q, k=3):
    from langchain.chains import RetrievalQA
    from langchain_openai import ChatOpenAI

    llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)

    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': k})

    chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
    
    answer = chain.invoke(q)
    return answer


### Running Code

#### Ask a PDF

In [11]:
data = load_document('files/us_constitution.pdf')

print(f'You have {len(data)} pages in your data')
print(f'There are {len(data[20].page_content)} characters in the page')

Loading files/us_constitution.pdf
You have 41 pages in your data
There are 1137 characters in the page


In [59]:
# data = load_document('files/the_great_gatsby.docx')
# print(data[0].page_content)

In [60]:
# data = load_from_wikipedia('GPT-4', 'de')
# print(data[0].page_content)

In [14]:
chunks = chunk_data(data)

print(len(chunks))

190


In [15]:
print_embedding_cost(chunks)

Total Tokens: 16711
Embedding Cost in USD: 334.220000


In [17]:
delete_pinecone_index()

Deleting all indexes ... 
Ok


In [18]:
index_name = 'askadocument'
vector_store = insert_or_fetch_embeddings(index_name=index_name, chunks=chunks)

Creating index askadocument and embeddings ...Ok


In [19]:
q = 'What is the Bill of Rights?'
answer = ask_and_get_answer(vector_store, q)

print(answer)

{'query': 'What is the Bill of Rights?', 'result': 'The Bill of Rights refers to the first ten amendments to the United States Constitution. These amendments were added to the Constitution in 1791 to guarantee certain rights and protections to the people.'}


#### While Loop for Asking Questions

In [20]:
import time

i = 1
print('Write Quit or Exit to quit.')

while True:
    q = input(f'Question #{i}: ')
    i = i + 1

    if q.lower() in ['quit', 'exit']:
        print('Quitting ... bye bye!')
        time.sleep(2)
        break
    
    answer = ask_and_get_answer(vector_store, q)

    print(f'\nAnswer: {answer}')
    print(f'\n {"-" * 50} \n')

Write Quit or Exit to quit.

Answer: {'query': 'What is the Bill of Rights?', 'result': 'The Bill of Rights is the first ten amendments to the United States Constitution. It outlines specific protections of individual liberties and limits the power of the government.'}

 -------------------------------------------------- 


Answer: {'query': 'What is the Bill of Rights?', 'result': 'The Bill of Rights is the first ten amendments to the United States Constitution. It includes rights such as freedom of speech, religion, and the right to bear arms, among others.'}

 -------------------------------------------------- 

Quitting ... bye bye!


#### Ask Wikipedia

In [21]:
delete_pinecone_index()

Deleting all indexes ... 
Ok


In [24]:
data = load_from_wikipedia('Google Gemini', 'de')
chunks = chunk_data(data)

In [25]:
chunks

[Document(metadata={'title': 'Google Gemini', 'summary': 'Google Gemini (ehemals Google Bard) ist ein von Google entwickelter KI-basierter, multimodaler Chatbot. Er wurde als direkte Reaktion auf den Erfolg von ChatGPT entwickelt und im März 2023 in eingeschränkter Kapazität veröffentlicht, bevor er im Laufe des Sommers in weiteren Ländern verfügbar wurde. Google Gemini ist in 40 Sprachen verfügbar.', 'source': 'https://de.wikipedia.org/wiki/Google_Gemini'}, page_content='Google Gemini (ehemals Google Bard) ist ein von Google entwickelter KI-basierter, multimodaler Chatbot. Er wurde als direkte Reaktion auf den Erfolg von ChatGPT entwickelt und im März 2023 in eingeschränkter Kapazität veröffentlicht, bevor er im Laufe des'),
 Document(metadata={'title': 'Google Gemini', 'summary': 'Google Gemini (ehemals Google Bard) ist ein von Google entwickelter KI-basierter, multimodaler Chatbot. Er wurde als direkte Reaktion auf den Erfolg von ChatGPT entwickelt und im März 2023 in eingeschränkte

In [26]:
index_name = 'gemini'
vector_store = insert_or_fetch_embeddings(index_name=index_name, chunks=chunks)

Creating index gemini and embeddings ...Ok


In [27]:
q = 'Was ist Google Gemini?'
answer = ask_and_get_answer(vector_store, q)

print(answer)

{'query': 'Was ist Google Gemini?', 'result': 'Ich kenne Google Gemini nicht.'}
